# Librerías

In [28]:
import time
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import warnings 
warnings.filterwarnings('ignore')

# Funciones

In [15]:
def sumar_numeros_rango(df, columna_texto):
    def extraer_y_sumar(text):
        numeros = re.findall(r'\d+(?:\.\d{1,2})?', text)
        numeros_en_rango = [float(num) for num in numeros if 50 <= float(num) <= 50000]
        return sum(numeros_en_rango) if numeros_en_rango else np.nan
    df['INGRESO'] = df[columna_texto].apply(lambda x: extraer_y_sumar(str(x)))
    
def get_embedding(texts):
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    model = SentenceTransformer(EMBEDDING_MODEL)
    embedding = model.encode(texts, convert_to_numpy=True) 
    return embedding 

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

# Limpieza de Datos

In [54]:
data = pd.read_csv('data/REPARACIONES.csv', encoding='latin1')
sumar_numeros_rango(data, 'INFORMETALLER')
data_cleaned = data.dropna(subset=['INGRESO'])

data_cleaned["COMBINADO"] = (
    data_cleaned["FALLA"].fillna(" ") + " " +
    data_cleaned["OBSERVACIONES"].fillna(" ") + " " +
    data_cleaned["INFORMETALLER"].fillna(" ") )

data_cleaned.reset_index(inplace=True)

data_cleaned["COMBINADO"] = (
    data_cleaned["COMBINADO"]
    .str.replace(r"[^a-zA-Z\s]", "", regex=True)
    .str.lower())
data_cleaned = data_cleaned[['CLIENTE', 'COMBINADO', 'INGRESO']]
data_cleaned.head(10)

,CLIENTE,COMBINADO,INGRESO
0,2,revision general tndr cambio del panel de cont...,1820.0
1,12,incluir el editor de pdf contasea hienz inc...,150.0
2,2,imprime negro tn mantenimiento iva,480.0
3,3,mayusculas no escribe minusculasse nota mas ...,360.0
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0
5,2,rev gral cartucho toner reparacion traccion de...,1030.0
6,2,rev gral sin toner con cable de energia bue...,530.0
7,2,rev gral sin toner cambio de contacto del ca...,480.0
8,3,rev gral toner reparacion de traccion de papel...,480.0
9,4,rev gral toners el juego de cartuchos que envi...,400.0


# Creación de Embeddings

In [55]:
keywords = {
    'Impresora Laser': ['impresora laser', 'tn', 'toner', 'drum', 'tndr',
                        'calor', 'presion', 'pad', 'termistor'],
    'Impresora Tinta': ['impresora tinta', 'reset', 'almohadillas', 'bateria',
                        '12v', 'cinta', 'perilla', 'cabezal', 'manguera', 'mascarilla'],
    'Sistemas': ['sistema', 'office', 'pantalla', 'windows', 'bisagra', 'teclado',
                 'mouse', 'mousepad', 'ssd', 'operativo', 'disco duro', 'mayusculas', 'minusculas',
                 'ram', 'procesador', 'madre', 'correo', 'gabinete', 'bios'],
    'Otros': ['otros']
}

In [49]:
inicio = time.time()
################################################################

imp_laser = []
imp_tinta = []
sistemas = []
otros = []

for palabra in keywords['Impresora Laser']:
    embedding = get_embedding(palabra)
    imp_laser.append(embedding)

for palabra in keywords['Impresora Tinta']:
    embedding = get_embedding(palabra)
    imp_tinta.append(embedding)

for palabra in keywords['Sistemas']:
    embedding = get_embedding(palabra)
    sistemas.append(embedding)

for palabra in keywords['Otros']:
    embedding = get_embedding(palabra)
    otros.append(embedding)
    
keywords_embedding = {
    'Impresora Laser':imp_laser,
    'Impresora Tinta':imp_tinta,
    'Sistemas':sistemas,
    'Otros':otros
}

################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 35.1624 segundos en ejecutarse.


In [70]:
inicio = time.time()
################################################################

muestra_embedding = []
palabras_embedding = []
muestra = data_cleaned['COMBINADO'].head(2).reset_index()

for i in range(len(muestra)):
    embedding = get_embedding(muestra['COMBINADO'][i])
    palabras = muestra['COMBINADO'][i].split()
    muestra_embedding.append(embedding)
    palabras_embedding.append(palabras)

################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 2.2175 segundos en ejecutarse.


In [81]:
len(palabras_embedding[1])

13

In [79]:
resultados_correlacion = []

for i, vector in enumerate(muestra_embedding):
    for key, lista_vectores in keywords_embedding.items():
        for j, vector_dict in enumerate(lista_vectores):
            cos_sim = cosine_similarity(vector, vector_dict)
            resultados_correlacion.append({
                "Número de falla": i + 1, 
                "Segmento": key,  
                "Palabra": j+1, 
                "Similitud Coseno": cos_sim
            })

df_resultados = pd.DataFrame(resultados_correlacion)

df_resultados.head(10)

,Número de falla,Segmento,Palabra,Similitud Coseno
0,1,Impresora Laser,1,0.100994
1,1,Impresora Laser,2,0.121243
2,1,Impresora Laser,3,0.068871
3,1,Impresora Laser,4,0.009358
4,1,Impresora Laser,5,0.298307
5,1,Impresora Laser,6,0.057049
6,1,Impresora Laser,7,0.220492
7,1,Impresora Laser,8,0.062818
8,1,Impresora Laser,9,0.137893
9,1,Impresora Tinta,1,0.188350


In [47]:
data_cleaned['COMBINADO'].head(1)

0    revision general tndr cambio panel control bue...
Name: COMBINADO, dtype: object